In [1]:
import pandas as pd

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, balanced_accuracy_score
from scipy.stats import pearsonr, spearmanr

# import os
# import ray
# ray.init()

# os.environ["RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE"] = "1"
# os.environ["MODIN_MEMORY"] = "10000000000"

# import modin.pandas as mpd
# import modin.config as cfg; print(cfg.CpuCount.get()); cfg.CpuCount.put(12); print(cfg.CpuCount.get())


# Increse column width to display the full content of the columns
pd.set_option('display.max_colwidth', 800)

In [3]:
# test set 

test = pd.read_json("../data/test-data/final_combined_test_data.json", lines=True)

all_scores_df = test.copy()

print(len(test))
test.label.value_counts()


from datasets import load_dataset
llm_aggrefact = load_dataset("lytang/LLM-AggreFact")
llm_aggrefact = llm_aggrefact['test'].to_pandas()

llm_aggrefact = llm_aggrefact.rename(columns={'doc': 'source', 'claim': 'target', 'label': 'label', 'dataset': 'subset'})

# replace 1 and 0 with Yes and No
def replace_label(row):
    label = row['label']
    if label == 1:
        row['label'] = 'Yes'
    else:
        row['label'] = 'No'
    return row
print(len(llm_aggrefact))
llm_aggrefact = llm_aggrefact.apply(replace_label, axis=1).drop_duplicates(subset=['source', 'target'])

y_aggrefact = llm_aggrefact[llm_aggrefact['subset'].isin( ["AggreFact-CNN", "AggreFact-XSum"])]
print( y_aggrefact['subset'].value_counts())

y_aggrefact = y_aggrefact[['source', 'target', 'label', 'subset']]

subsets_llm_aggrefact = ["tofueval_meetingbank", "tofueval_mediasum", "WiCE",\
    "ExpertQA", "Reveal", "FactCheck-GPT", "ClaimVerify", "Lfqa"]

y_aggrefact.sample(2)

62823


Generating dev split:   0%|          | 0/15080 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12949 [00:00<?, ? examples/s]

12949
subset
AggreFact-CNN     558
AggreFact-XSum    558
Name: count, dtype: int64


,source,target,label,subset
1,"tom o'carroll , a leading member of the notorious paedophile information exchange ( above ) , has joined the supporters of the media pressure group hacked off\na leading member of the notorious paedophile information exchange , which campaigned to legalise child sex in the 1970s and 80s , has joined the supporters of the media pressure group hacked off .\ntom o'carroll , a former open university information officer and formerly a key activist for pie , attended a hacked off rally in the houses of parliament on february 25 to lobby mps for state involvement in media regulation .\nthe organisation , set up in the wake of the phone-hacking scandal , is campaigning against what it sees as the ` biased and unfair ' independent press standards organisation .\nit wants mps to set up a statuto...",tom o'carroll is a former open university information officer and formerly a key activist for pie . \nhe attended a hacked off rally in the houses of parliament on february 25 to lobby mps for state involvement in media regulation . \nthe organisation is campaigning against what it sees as the ` biased and unfair ` independent press standards organisation .,Yes,AggreFact-CNN
623,"The review into 22 abuse cases in Northern Ireland criticised the authorities for not doing enough.\nThe young people went missing a number of times over a 20-month period while being looked after in the care system.\nThe Police Service of Northern Ireland (PSNI) said it has recently made policy changes aimed at keeping children safe.\nIn September 2013, the PSNI said it had begun a major investigation into the sexual exploitation of children and young people who had gone missing from care in Northern Ireland.\nOfficers said they had identified 22 people, aged between 13 and 18, who may have been sexually exploited.\nThe PSNI investigation was known was Operation Owl.\nThursday's report, examining the PSNI response, has been published by the Safeguarding Board for Northern Ireland.\nIt...","A police investigation into the sexual exploitation of teenagers who went missing from care was `` limited and inconsistent'', a report has found.",No,AggreFact-XSum


### Summary statistics

In [4]:
# Print subset sizes
for subset in test.subset.unique():
    print(f"{subset}: {len(test[test.subset == subset])}")

aggrefact_cnndm: 1017
aggrefact_xsum: 1335
bump_cnndm: 785
halueval_cnndm: 19998
fib_xsum: 3534
fib_cnndm: 543
llm_summaries_cnndm: 1829
llm_summaries_xsum: 1726
gumsum_gumsum: 95
instrusum_instrusum: 458
tofueval_mediasum: 725
tofueval_meetingbank: 770
diasumfact_QMSum: 569
diasumfact_SAMSum: 669
anli: 3200
alisawuffles/WANLI: 5000
scitail: 2126
DeFacto: 1836
DADC-NLI: 766
FoolMeTwice: 1379
WiCE: 358
Seahorse: 4135
Reveal: 1705
ClaimVerify: 1087
FactCheck-GPT: 1565
ExpertQA: 3702
Lfqa: 1911


In [5]:
# Percentage of "No" in each subset

for subset in test.subset.unique():
    print(f"{subset}: {test[test.subset == subset].label.value_counts(normalize=True)}")

aggrefact_cnndm: label
Yes    0.885939
No     0.114061
Name: proportion, dtype: float64
aggrefact_xsum: label
Yes    0.508614
No     0.491386
Name: proportion, dtype: float64
bump_cnndm: label
No     0.873885
Yes    0.126115
Name: proportion, dtype: float64
halueval_cnndm: label
No     0.50005
Yes    0.49995
Name: proportion, dtype: float64
fib_xsum: label
No     0.858517
Yes    0.141483
Name: proportion, dtype: float64
fib_cnndm: label
No     0.823204
Yes    0.176796
Name: proportion, dtype: float64
llm_summaries_cnndm: label
Yes    0.911974
No     0.088026
Name: proportion, dtype: float64
llm_summaries_xsum: label
Yes    0.767092
No     0.232908
Name: proportion, dtype: float64
gumsum_gumsum: label
Yes    0.621053
No     0.378947
Name: proportion, dtype: float64
instrusum_instrusum: label
Yes    0.759825
No     0.240175
Name: proportion, dtype: float64
tofueval_mediasum: label
Yes    0.762759
No     0.237241
Name: proportion, dtype: float64
tofueval_meetingbank: label
Yes    0.805195

In [6]:
def get_length(row):
    source = row['source']
    target = row['target']
    
    row['length_source'] = len(source.split())
    row['length_target'] = len(target.split())
    
    return row

test = test.apply(get_length, axis=1)

print("Test set size: ", len(test))
print("Average source length: ", test.length_source.mean())
print("Average target length: ", test.length_target.mean())


Test set size:  62823
Average source length:  413.05130286678445
Average target length:  34.68107221877338


In [7]:
# Print subset sizes and average lengths for each subset

for subset in test.subset.unique():
    subset_data = test[test.subset == subset]
    print(subset)
    print("Size: ", len(subset_data))
    print("Average source length: ", subset_data.length_source.mean())
    print("Average target length: ", subset_data.length_target.mean())
    print()

aggrefact_cnndm
Size:  1017
Average source length:  498.0540806293019
Average target length:  54.52015732546706

aggrefact_xsum
Size:  1335
Average source length:  324.84044943820226
Average target length:  23.283146067415732

bump_cnndm
Size:  785
Average source length:  696.5031847133758
Average target length:  52.085350318471335

halueval_cnndm
Size:  19998
Average source length:  663.1723172317231
Average target length:  60.84253425342534

fib_xsum
Size:  3534
Average source length:  231.25070741369552
Average target length:  20.475099037917374

fib_cnndm
Size:  543
Average source length:  390.81767955801104
Average target length:  61.60957642725599

llm_summaries_cnndm
Size:  1829
Average source length:  458.3428102788409
Average target length:  69.32094040459268

llm_summaries_xsum
Size:  1726
Average source length:  307.41077636152954
Average target length:  25.01332560834299

gumsum_gumsum
Size:  95
Average source length:  777.0842105263158
Average target length:  27.0631578947

### FactAlign

In [8]:
base_bf16 = pd.read_csv('../data/eval-data/base-bf16.csv')
large_bf16 = pd.read_csv('../data/eval-data/large-bf16.csv')


print(len(base_bf16))
print(len(large_bf16))
print(base_bf16.label.value_counts())
print(large_bf16.label.value_counts())
print(base_bf16.subset.value_counts())
print(large_bf16.subset.value_counts())
print(base_bf16.columns)
print(large_bf16.columns)



62823
62823
label
Yes    31469
No     31354
Name: count, dtype: int64
label
Yes    31469
No     31354
Name: count, dtype: int64
subset
halueval_cnndm          19998
alisawuffles/wanli       5000
seahorse                 4135
expertqa                 3702
fib_xsum                 3534
anli                     3200
scitail                  2126
lfqa                     1911
defacto                  1836
llm_summaries_cnndm      1829
llm_summaries_xsum       1726
reveal                   1705
factcheck-gpt            1565
foolmetwice              1379
aggrefact_xsum           1335
claimverify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
dadc-nli                  766
tofueval_mediasum         725
diasumfact_samsum         669
diasumfact_qmsum          569
fib_cnndm                 543
instrusum_instrusum       458
wice                      358
gumsum_gumsum              95
Name: count, dtype: int64
subset
halueval_cnndm        

In [9]:
def get_source_target(text):
    
    source = text.split('_target_')[1]
    target = source.split('_source_')[0].strip()
    source = source.split('_source_')[1].strip()
    
    return source, target

# get_source_target(base_bf16.iloc[0]['input'])

new_base_bf16 = base_bf16.copy()
new_large_bf16 = large_bf16.copy()

new_base_bf16['source'], new_base_bf16['target'] = zip(*new_base_bf16['input'].map(get_source_target))

In [10]:
x = new_base_bf16.drop_duplicates(subset=['source', 'target'], inplace=False)
len(x)

62645

In [11]:
print(base_bf16.label.value_counts(), base_bf16.preds.value_counts())
print(large_bf16.label.value_counts(), large_bf16.preds.value_counts())
base_bf16.sample(2), large_bf16.sample(2)

label
Yes    31469
No     31354
Name: count, dtype: int64 preds
Yes    36930
No     25893
Name: count, dtype: int64
label
Yes    31469
No     31354
Name: count, dtype: int64 preds
Yes    37532
No     25291
Name: count, dtype: int64


(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 input  \
 61002  Can the target text be inferred from the source text? In other words, is the target text factually consistent with the source text? Answer with a "Yes" or "No". _target_ It also men

In [12]:
# Invert probs if preds = No

def invert_prob(row):
    preds = row['preds']
    label_prob = row['label_prob']
    if preds == 'No':
        row['label_prob'] = 1 - label_prob
    return row

base_bf16 = base_bf16.apply(invert_prob, axis=1)
base_bf16.head(2)
large_bf16 = large_bf16.apply(invert_prob, axis=1)
large_bf16.head(2)

,input,label,preds,subset,label_prob
0,"Can the target text be inferred from the source text? In other words, is the target text factually consistent with the source text? Answer with a ""Yes"" or ""No"". _target_ Marseille prosecutor says ""so far no videos were used in the crash investigation"" despite media reports. Journalists at Bild and Paris Match are ""very confident"" the video clip is real, an editor says. Andreas Lubitz had informed his Lufthansa training school of an episode of severe depression, airline says. _source_ Marseille, France (CNN)The French prosecutor leading an investigation into the crash of Germanwings Flight 9525 insisted Wednesday that he was not aware of any video footage from on board the plane. Marseille prosecutor Brice Robin told CNN that ""so far no videos were used in the crash investigation."" He a...",Yes,Yes,halueval_cnndm,0.998905
1,"Can the target text be inferred from the source text? In other words, is the target text factually consistent with the source text? Answer with a ""Yes"" or ""No"". _target_ A video showing the final moments of Germanwings Flight 9525 has been recovered by investigators from the wreckage site. Marseille prosecutor Brice Robin urged anyone who might have more footage to turn it over immediately. Andreas Lubitz, the co-pilot accused of deliberately crashing the plane, had a history of severe depression and suicidal tendencies. _source_ Marseille, France (CNN)The French prosecutor leading an investigation into the crash of Germanwings Flight 9525 insisted Wednesday that he was not aware of any video footage from on board the plane. Marseille prosecutor Brice Robin told CNN that ""so far no vid...",No,No,halueval_cnndm,0.096444


In [13]:
# Change "No" to 0 and "Yes" to 1

def change(row):
    label = row['label']
    prediction = row['preds']
    
    if label == 'No':
        row['label'] = 0
    if label == 'Yes':
        row['label'] = 1
    
    if prediction == 'No':
        row['preds'] = 0
    if prediction == 'Yes':
        row['preds'] = 1
        
    return row

base_bf16 = base_bf16.apply(change, axis=1)
large_bf16 = large_bf16.apply(change, axis=1)

In [14]:
# For each subset calculate the AUC ROC score

def calculate_auc(df):
    return roc_auc_score(df['label'], df['label_prob'])



base_bf16_auc = base_bf16.groupby('subset').apply(calculate_auc)
base_bf16_auc

subset
aggrefact_cnndm         0.685206
aggrefact_xsum          0.740744
alisawuffles/wanli      0.877220
anli                    0.753980
bump_cnndm              0.685735
claimverify             0.799364
dadc-nli                0.861109
defacto                 0.773372
diasumfact_qmsum        0.700476
diasumfact_samsum       0.756583
expertqa                0.594624
factcheck-gpt           0.822948
fib_cnndm               0.271905
fib_xsum                0.820808
foolmetwice             0.918891
gumsum_gumsum           0.814972
halueval_cnndm          0.598213
instrusum_instrusum     0.574425
lfqa                    0.885055
llm_summaries_cnndm     0.843179
llm_summaries_xsum      0.757090
reveal                  0.900094
scitail                 0.984603
seahorse                0.720948
tofueval_mediasum       0.781425
tofueval_meetingbank    0.766602
wice                    0.865959
dtype: float64

In [15]:
large_bf16_auc = large_bf16.groupby('subset').apply(calculate_auc)
large_bf16_auc

subset
aggrefact_cnndm         0.696788
aggrefact_xsum          0.748482
alisawuffles/wanli      0.889535
anli                    0.833812
bump_cnndm              0.769488
claimverify             0.799725
dadc-nli                0.899190
defacto                 0.807595
diasumfact_qmsum        0.623149
diasumfact_samsum       0.759011
expertqa                0.614970
factcheck-gpt           0.840107
fib_cnndm               0.546304
fib_xsum                0.875578
foolmetwice             0.940031
gumsum_gumsum           0.827213
halueval_cnndm          0.673818
instrusum_instrusum     0.556923
lfqa                    0.881214
llm_summaries_cnndm     0.854328
llm_summaries_xsum      0.758234
reveal                  0.904887
scitail                 0.992728
seahorse                0.744268
tofueval_mediasum       0.754100
tofueval_meetingbank    0.758817
wice                    0.887843
dtype: float64

In [23]:
subsets_ = ['reveal', 'tofueval_mediasum', 'tofueval_meetingbank', 'wice', 'expertqa', 'claimverify', 'factcheck-gpt', 'lfqa']

In [24]:
base_bf16_llm_aggrefact = base_bf16[base_bf16['subset'].isin(subsets_)]
large_bf16_llm_aggrefact = large_bf16[large_bf16['subset'].isin(subsets_)]

y_base_bf16 = pd.read_csv("../data/eval-data/base-bf16-aggrefact-llm.csv", )
y_large_bf16 = pd.read_csv("../data/eval-data/large-bf16-aggrefact-llm.csv", )


y_base_bf16 = y_base_bf16.apply(change, axis=1)
y_large_bf16 = y_large_bf16.apply(change, axis=1)


base_bf16_llm_aggrefact = pd.concat([base_bf16_llm_aggrefact, y_base_bf16], axis=0)
large_bf16_llm_aggrefact = pd.concat([large_bf16_llm_aggrefact, y_large_bf16], axis=0)

In [25]:
base_bf16_llm_aggrefact.subset.value_counts(), large_bf16_llm_aggrefact.subset.value_counts()

(subset
 expertqa                3702
 lfqa                    1911
 reveal                  1705
 factcheck-gpt           1565
 claimverify             1087
 tofueval_meetingbank     770
 tofueval_mediasum        725
 aggrefact-cnn            558
 aggrefact-xsum           558
 wice                     358
 Name: count, dtype: int64,
 subset
 expertqa                3702
 lfqa                    1911
 reveal                  1705
 factcheck-gpt           1565
 claimverify             1087
 tofueval_meetingbank     770
 tofueval_mediasum        725
 aggrefact-xsum           558
 aggrefact-cnn            558
 wice                     358
 Name: count, dtype: int64)

In [26]:
base_bf16_llm_aggrefact.label.value_counts(), base_bf16_llm_aggrefact.preds.value_counts(), large_bf16_llm_aggrefact.label.value_counts(), large_bf16_llm_aggrefact.preds.value_counts()

(label
 1    7725
 0    5214
 Name: count, dtype: int64,
 preds
 1    6857
 0    6082
 Name: count, dtype: int64,
 label
 1    7725
 0    5214
 Name: count, dtype: int64,
 preds
 1    7474
 0    5465
 Name: count, dtype: int64)

In [28]:
acc = 0
n = 0

for subset in base_bf16_llm_aggrefact.subset.unique():
    subset_data = base_bf16_llm_aggrefact[base_bf16_llm_aggrefact.subset == subset]
    score = balanced_accuracy_score(subset_data['label'].to_list(), subset_data['preds'].to_list())
    acc += score
    size = len(subset_data)
    print(subset, size, score)
    n += 1

print("Average balanced accuracy: ", acc/n)
print()
acc = 0
n = 0

for subset in large_bf16_llm_aggrefact.subset.unique():
    subset_data = large_bf16_llm_aggrefact[large_bf16_llm_aggrefact.subset == subset]
    score = balanced_accuracy_score(subset_data['label'].to_list(), subset_data['preds'].to_list())
    acc += score
    size = len(subset_data)
    print(subset, size, score)
    n += 1
    
    
print("Average balanced accuracy: ", acc/n)

tofueval_mediasum 725 0.68153622944615
factcheck-gpt 1565 0.7334990068536048
expertqa 3702 0.5668986246898311
claimverify 1087 0.7107650496338072
lfqa 1911 0.8071884280536139
wice 358 0.7658751869278185
tofueval_meetingbank 770 0.7093548387096774
reveal 1705 0.798525911885364
aggrefact-cnn 558 0.5734846097279126
aggrefact-xsum 558 0.6815693078850973
Average balanced accuracy:  0.7028697193812877

claimverify 1087 0.7405070559113991
expertqa 3702 0.5894884015616532
tofueval_mediasum 725 0.6987152529542874
tofueval_meetingbank 770 0.6860215053763441
reveal 1705 0.8100544805127396
wice 358 0.7998869314658787
factcheck-gpt 1565 0.7721310595351001
lfqa 1911 0.8011410472114635
aggrefact-xsum 558 0.6903412377096587
aggrefact-cnn 558 0.5775291522218721
Average balanced accuracy:  0.7165816124460396


### ROUGE

In [22]:

import evaluate
import numpy as np
rouge = evaluate.load('rouge')

import warnings
warnings.filterwarnings('ignore')

test_set = pd.read_json("../data/test-data/final_combined_test_data.json", lines=True)

In [23]:

def calculate_rouge(df):
    predictions = df['target'].to_list()
    references = df['source'].to_list()
    r = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
    
    rouge1 = r['rouge1']
    rouge2 = r['rouge2']
    rougeL = r['rougeL']
    
    print(f"Aggregate ROUGE-1: {np.mean(rouge1)}\nAggregate ROUGE-2: {np.mean(rouge2)}\nAggregate ROUGE-L: {np.mean(rougeL)}\n")
    
    return r

new_df = pd.DataFrame(columns=['subset', 'source', 'target', 'rouge1', 'rouge2', 'rougeL', 'label'])
# For each subset calculate the ROUGE scores
for subset in test_set['subset'].unique():
    if not subset:
        continue
    print(subset)
    subset_df = test_set[test_set['subset'] == subset]
    print(f"Length: {len(subset_df)}")
    results = calculate_rouge(subset_df)
    
    rouge1 = results['rouge1']
    rouge2 = results['rouge2']
    rougeL = results['rougeL']
    
    subset_df['rouge1'] = rouge1
    subset_df['rouge2'] = rouge2
    subset_df['rougeL'] = rougeL
    
    new_df = pd.concat([new_df, subset_df])
    
    # print(results)

aggrefact_cnndm
Length: 1017
Aggregate ROUGE-1: 0.21336238081138642
Aggregate ROUGE-2: 0.18292301039355094
Aggregate ROUGE-L: 0.18967448228850625

aggrefact_xsum
Length: 1335
Aggregate ROUGE-1: 0.12337285704193096
Aggregate ROUGE-2: 0.04194765176067925
Aggregate ROUGE-L: 0.08243328944798624

bump_cnndm
Length: 785
Aggregate ROUGE-1: 0.13706340423598337
Aggregate ROUGE-2: 0.0759082226851162
Aggregate ROUGE-L: 0.09803528013557283

halueval_cnndm
Length: 19998
Aggregate ROUGE-1: 0.16267543526898423
Aggregate ROUGE-2: 0.08451953190630239
Aggregate ROUGE-L: 0.11249894803312982

fib_xsum
Length: 3534
Aggregate ROUGE-1: 0.12062514152347291
Aggregate ROUGE-2: 0.03506305169123534
Aggregate ROUGE-L: 0.08082083106524639

fib_cnndm
Length: 543
Aggregate ROUGE-1: 0.2874173456871638
Aggregate ROUGE-2: 0.2641649319649293
Aggregate ROUGE-L: 0.26786481912216975

llm_summaries_cnndm
Length: 1829
Aggregate ROUGE-1: 0.24071999641651523
Aggregate ROUGE-2: 0.1988684248408106
Aggregate ROUGE-L: 0.21488670276

In [24]:
len(test_set), len(new_df)

(62823, 62823)

In [25]:
new_df.to_csv("../data/eval-data/rouge.csv", sep="\t", index=False)
new_df.head(2)

,subset,source,target,rouge1,rouge2,rougeL,label
0,aggrefact_cnndm,"looking after elderly parents can be difficult at the best of times .\nbut this man takes caring for his alzheimer 's - suffering mother to another level .\na security guard from china has touched hearts across the country because he takes his 84-year-old mother with him to work on the back of his motorbike every single day , reported the people 's daily online .\nlu xincai , who lives in zhejiang province in eastern china , says that he is scared his mother will get lost if he leaves her at home by herself because she suffers from the degenerative disease .\ndevoted : lu xincai takes his 84-year-old mother to work with him on the back of his motorbike every day .\nhe ties a sash around both of their waists to make sure she does n't fall off\nshe would often go up to the mountains to c...",lu xincai takes his 84-year-old mother to work with him on the back of his motorbike every day . \nhis mother suffers from alzheimer 's and used to get lost when she was left alone . \nhe ties a sash around both of their waists to ensure that she does not fall off .,0.224101,0.216561,0.177590,Yes
1,aggrefact_cnndm,"tokyo ( cnn ) a bizarre and alarming discovery is raising concerns in japan about the potential for terrorism involving drones .\na drone carrying traces of a radioactive material was found on the rooftop of japan 's equivalent to the white house on wednesday , police and government officials said .\nthe discovery came on the same day a japanese court approved a government plan to restart two reactors at the sendai nuclear power plant in kagoshima prefecture , more than four years after the fukushima daiichi nuclear disaster .\nprime minister shinzo abe 's push to restart the reactors is unpopular among many japanese , who view nuclear energy as too dangerous .\na staff member spotted the drone wednesday morning on the roof of abe 's residence , tokyo metropolitan police said .\ndozens...","a drone carrying traces of a radioactive material was found on the rooftop of japan 's equivalent to the white house .\nthe discovery came on the same day a japanese court approved a government plan to restart two reactors at the sendai nuclear power plant . \nthe drone was equipped with a small camera , smoke flares and a bottle containing small traces of cesium .",0.300716,0.278177,0.300716,Yes


In [16]:
new_df = pd.read_csv("../data/eval-data/rouge.csv", sep="\t")

new_df['label'] = new_df['label'].apply(lambda x: 0 if x == 'No' else 1) # convert label "Yes" into 1 and "No" into 0

In [17]:
rouge_type = ['rouge1', 'rouge2', 'rougeL'][2]

# calculate auc
def calculate_auc(df):
    return roc_auc_score(df['label'], df[rouge_type])


rouge_auc = new_df.groupby('subset').apply(calculate_auc)

print(rouge_auc)

subset
ClaimVerify             0.533415
DADC-NLI                0.440839
DeFacto                 0.508740
ExpertQA                0.514609
FactCheck-GPT           0.568482
FoolMeTwice             0.523795
Lfqa                    0.772828
Reveal                  0.608176
Seahorse                0.509130
WiCE                    0.500036
aggrefact_cnndm         0.691449
aggrefact_xsum          0.470915
alisawuffles/WANLI      0.512884
anli                    0.452784
bump_cnndm              0.518678
diasumfact_QMSum        0.561600
diasumfact_SAMSum       0.571645
fib_cnndm               0.047271
fib_xsum                0.386898
gumsum_gumsum           0.743409
halueval_cnndm          0.424876
instrusum_instrusum     0.456635
llm_summaries_cnndm     0.852697
llm_summaries_xsum      0.600992
scitail                 0.751793
tofueval_mediasum       0.564448
tofueval_meetingbank    0.672796
dtype: float64


### QuestEval

In [18]:
questeval = pd.read_json("../data/eval-data/questeval.json", lines=True)
print(questeval.columns)
print(questeval.label.value_counts())
print(questeval.subset.value_counts())
print(len(questeval))
questeval.head(2)

Index(['source', 'target', 'label', 'subset', 'questeval'], dtype='object')
label
Yes    31469
No     31354
Name: count, dtype: int64
subset
halueval_cnndm          19998
alisawuffles/WANLI       5000
Seahorse                 4135
ExpertQA                 3702
fib_xsum                 3534
anli                     3200
scitail                  2126
Lfqa                     1911
DeFacto                  1836
llm_summaries_cnndm      1829
llm_summaries_xsum       1726
Reveal                   1705
FactCheck-GPT            1565
FoolMeTwice              1379
aggrefact_xsum           1335
ClaimVerify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
DADC-NLI                  766
tofueval_mediasum         725
diasumfact_SAMSum         669
diasumfact_QMSum          569
fib_cnndm                 543
instrusum_instrusum       458
WiCE                      358
gumsum_gumsum              95
Name: count, dtype: int64
62823


,source,target,label,subset,questeval
0,"looking after elderly parents can be difficult at the best of times .\nbut this man takes caring for his alzheimer 's - suffering mother to another level .\na security guard from china has touched hearts across the country because he takes his 84-year-old mother with him to work on the back of his motorbike every single day , reported the people 's daily online .\nlu xincai , who lives in zhejiang province in eastern china , says that he is scared his mother will get lost if he leaves her at home by herself because she suffers from the degenerative disease .\ndevoted : lu xincai takes his 84-year-old mother to work with him on the back of his motorbike every day .\nhe ties a sash around both of their waists to make sure she does n't fall off\nshe would often go up to the mountains to c...",lu xincai takes his 84-year-old mother to work with him on the back of his motorbike every day . \nhis mother suffers from alzheimer 's and used to get lost when she was left alone . \nhe ties a sash around both of their waists to ensure that she does not fall off .,Yes,aggrefact_cnndm,0.572732
1,"tokyo ( cnn ) a bizarre and alarming discovery is raising concerns in japan about the potential for terrorism involving drones .\na drone carrying traces of a radioactive material was found on the rooftop of japan 's equivalent to the white house on wednesday , police and government officials said .\nthe discovery came on the same day a japanese court approved a government plan to restart two reactors at the sendai nuclear power plant in kagoshima prefecture , more than four years after the fukushima daiichi nuclear disaster .\nprime minister shinzo abe 's push to restart the reactors is unpopular among many japanese , who view nuclear energy as too dangerous .\na staff member spotted the drone wednesday morning on the roof of abe 's residence , tokyo metropolitan police said .\ndozens...","a drone carrying traces of a radioactive material was found on the rooftop of japan 's equivalent to the white house .\nthe discovery came on the same day a japanese court approved a government plan to restart two reactors at the sendai nuclear power plant . \nthe drone was equipped with a small camera , smoke flares and a bottle containing small traces of cesium .",Yes,aggrefact_cnndm,0.508080


In [19]:
def convert_label(row):
    label = row['label']
    if label == 'Yes':
        row['label'] = 1
    if label == 'No':
        row['label'] = 0
    return row

questeval = questeval.apply(convert_label, axis=1)

roc = roc_auc_score(questeval['label'], questeval['questeval'])
print("AUC ROC score: ", roc)

# Subset wise AUC ROC score
questeval_auc = questeval.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['questeval']))
questeval_auc

AUC ROC score:  0.6240841791239211


subset
ClaimVerify             0.712643
DADC-NLI                0.568126
DeFacto                 0.645085
ExpertQA                0.593886
FactCheck-GPT           0.754832
FoolMeTwice             0.684465
Lfqa                    0.806550
Reveal                  0.849062
Seahorse                0.631419
WiCE                    0.599519
aggrefact_cnndm         0.714111
aggrefact_xsum          0.610757
alisawuffles/WANLI      0.623556
anli                    0.480432
bump_cnndm              0.620373
diasumfact_QMSum        0.569502
diasumfact_SAMSum       0.568736
fib_cnndm               0.342142
fib_xsum                0.611612
gumsum_gumsum           0.793785
halueval_cnndm          0.580613
instrusum_instrusum     0.540282
llm_summaries_cnndm     0.872060
llm_summaries_xsum      0.772084
scitail                 0.926951
tofueval_mediasum       0.663663
tofueval_meetingbank    0.706323
dtype: float64

### BARTScore



In [20]:
bartscore = pd.read_json("../data/eval-data/bartscore.json", lines=True)
print(bartscore.columns)
print(bartscore.label.value_counts())
print(bartscore.subset.value_counts())
print(len(bartscore))
bartscore.head(2)

Index(['source', 'target', 'label', 'subset', 'bartscore'], dtype='object')
label
Yes    31469
No     31354
Name: count, dtype: int64
subset
halueval_cnndm          19998
alisawuffles/WANLI       5000
Seahorse                 4135
ExpertQA                 3702
fib_xsum                 3534
anli                     3200
scitail                  2126
Lfqa                     1911
DeFacto                  1836
llm_summaries_cnndm      1829
llm_summaries_xsum       1726
Reveal                   1705
FactCheck-GPT            1565
FoolMeTwice              1379
aggrefact_xsum           1335
ClaimVerify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
DADC-NLI                  766
tofueval_mediasum         725
diasumfact_SAMSum         669
diasumfact_QMSum          569
fib_cnndm                 543
instrusum_instrusum       458
WiCE                      358
gumsum_gumsum              95
Name: count, dtype: int64
62823


,source,target,label,subset,bartscore
0,"It is important to keep in mind that in most cases, this problem is caused by the absence of an organization that can lead the way.",The absence of an organization that can lead the way is the cause of the problem.,No,alisawuffles/WANLI,-1.228808
1,"Paul Casey insists he is desperate to play in the Ryder Cup again, even though he looks unlikely to rejoin the European Tour in time to earn qualifying points this season. Only Tour members can play on the team and Phoenix-based Casey gave up his membership after deciding in January to concentrate on the PGA Tour in order to get back into the world's top 50. The move has paid off with the former world No 3 losing a play-off in the Northern Trust Open and finishing joint third in the Honda Classic to climb to 44th at the start of this week. Paul Casey, practicing ahead of the Masters in Augusta, 'desperately' wants to return to the Ryder Cup. The Englishman has returned to the world's top 50 after leaving the European Tour to concentrate on his game. However, even if he rejoins the Euro...",Paul Casey left the European Tour to get back into world's top 50. Former world No 3 is 'desperate' to return to the 2016 Ryder Cup team. Captain Darren Clarke has said he wants Casey at Hazeltine next year.\n,Yes,halueval_cnndm,-2.178005


In [21]:
bartscore = bartscore.apply(convert_label, axis=1)
roc = roc_auc_score(bartscore['label'], bartscore['bartscore'])

print("AUC ROC score: ", roc)

# Subset wise AUC ROC score
bartscore_auc = bartscore.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['bartscore']))
bartscore_auc

AUC ROC score:  0.569124651181143


subset
ClaimVerify             0.776601
DADC-NLI                0.497401
DeFacto                 0.584190
ExpertQA                0.578560
FactCheck-GPT           0.704020
FoolMeTwice             0.605546
Lfqa                    0.877772
Reveal                  0.792830
Seahorse                0.589391
WiCE                    0.634789
aggrefact_cnndm         0.657526
aggrefact_xsum          0.618682
alisawuffles/WANLI      0.577650
anli                    0.452429
bump_cnndm              0.583385
diasumfact_QMSum        0.666998
diasumfact_SAMSum       0.632761
fib_cnndm               0.020554
fib_xsum                0.441779
gumsum_gumsum           0.689266
halueval_cnndm          0.389579
instrusum_instrusum     0.570977
llm_summaries_cnndm     0.872056
llm_summaries_xsum      0.727802
scitail                 0.876055
tofueval_mediasum       0.682640
tofueval_meetingbank    0.712495
dtype: float64

### BERTScore

In [22]:
bertscore = pd.read_json("../data/eval-data/bertscore.json", lines=True)
print(bertscore.columns)
print(bertscore.label.value_counts())
print(bertscore.subset.value_counts())
print(len(bertscore))
bertscore.head(2)

Index(['source', 'target', 'label', 'subset', 'bertscore'], dtype='object')
label
Yes    31469
No     31354
Name: count, dtype: int64
subset
halueval_cnndm          19998
alisawuffles/WANLI       5000
Seahorse                 4135
ExpertQA                 3702
fib_xsum                 3534
anli                     3200
scitail                  2126
Lfqa                     1911
DeFacto                  1836
llm_summaries_cnndm      1829
llm_summaries_xsum       1726
Reveal                   1705
FactCheck-GPT            1565
FoolMeTwice              1379
aggrefact_xsum           1335
ClaimVerify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
DADC-NLI                  766
tofueval_mediasum         725
diasumfact_SAMSum         669
diasumfact_QMSum          569
fib_cnndm                 543
instrusum_instrusum       458
WiCE                      358
gumsum_gumsum              95
Name: count, dtype: int64
62823


,source,target,label,subset,bertscore
0,"looking after elderly parents can be difficult at the best of times .\nbut this man takes caring for his alzheimer 's - suffering mother to another level .\na security guard from china has touched hearts across the country because he takes his 84-year-old mother with him to work on the back of his motorbike every single day , reported the people 's daily online .\nlu xincai , who lives in zhejiang province in eastern china , says that he is scared his mother will get lost if he leaves her at home by herself because she suffers from the degenerative disease .\ndevoted : lu xincai takes his 84-year-old mother to work with him on the back of his motorbike every day .\nhe ties a sash around both of their waists to make sure she does n't fall off\nshe would often go up to the mountains to c...",lu xincai takes his 84-year-old mother to work with him on the back of his motorbike every day . \nhis mother suffers from alzheimer 's and used to get lost when she was left alone . \nhe ties a sash around both of their waists to ensure that she does not fall off .,Yes,aggrefact_cnndm,0.598911
1,"tokyo ( cnn ) a bizarre and alarming discovery is raising concerns in japan about the potential for terrorism involving drones .\na drone carrying traces of a radioactive material was found on the rooftop of japan 's equivalent to the white house on wednesday , police and government officials said .\nthe discovery came on the same day a japanese court approved a government plan to restart two reactors at the sendai nuclear power plant in kagoshima prefecture , more than four years after the fukushima daiichi nuclear disaster .\nprime minister shinzo abe 's push to restart the reactors is unpopular among many japanese , who view nuclear energy as too dangerous .\na staff member spotted the drone wednesday morning on the roof of abe 's residence , tokyo metropolitan police said .\ndozens...","a drone carrying traces of a radioactive material was found on the rooftop of japan 's equivalent to the white house .\nthe discovery came on the same day a japanese court approved a government plan to restart two reactors at the sendai nuclear power plant . \nthe drone was equipped with a small camera , smoke flares and a bottle containing small traces of cesium .",Yes,aggrefact_cnndm,0.637625


In [23]:
bertscore = bertscore.apply(convert_label, axis=1)
roc = roc_auc_score(bertscore['label'], bertscore['bertscore'])

print("AUC ROC score: ", roc)

# Subset wise AUC ROC score
bertscore_auc = bertscore.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['bertscore']))
bertscore_auc

AUC ROC score:  0.5082214167791583


subset
ClaimVerify             0.704702
DADC-NLI                0.438038
DeFacto                 0.573767
ExpertQA                0.550602
FactCheck-GPT           0.726649
FoolMeTwice             0.574640
Lfqa                    0.855923
Reveal                  0.662800
Seahorse                0.564131
WiCE                    0.558595
aggrefact_cnndm         0.713403
aggrefact_xsum          0.551644
alisawuffles/WANLI      0.556264
anli                    0.424766
bump_cnndm              0.543776
diasumfact_QMSum        0.660839
diasumfact_SAMSum       0.625868
fib_cnndm               0.124278
fib_xsum                0.544288
gumsum_gumsum           0.759416
halueval_cnndm          0.523280
instrusum_instrusum     0.523041
llm_summaries_cnndm     0.887450
llm_summaries_xsum      0.684115
scitail                 0.800671
tofueval_mediasum       0.706043
tofueval_meetingbank    0.685129
dtype: float64

### gpt-3.5-turbo

In [24]:
gpt = pd.read_json("../data/eval-data/gpt3.5.json", lines=True)
print(len(gpt))
gpt = gpt.drop_duplicates(subset=['source', 'target'])
print(len(gpt))
print(gpt.columns)
print(gpt.label.value_counts())
print(gpt.subset.value_counts())

63320
62823
Index(['source', 'target', 'label', 'subset', 'prediction', 'prob'], dtype='object')
label
Yes    31469
No     31354
Name: count, dtype: int64
subset
halueval_cnndm          19998
alisawuffles/WANLI       5000
Seahorse                 4136
ExpertQA                 3702
fib_xsum                 3533
anli                     3200
scitail                  2126
Lfqa                     1911
DeFacto                  1841
llm_summaries_cnndm      1829
llm_summaries_xsum       1725
Reveal                   1705
FactCheck-GPT            1565
FoolMeTwice              1379
aggrefact_xsum           1331
ClaimVerify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
DADC-NLI                  766
tofueval_mediasum         725
diasumfact_SAMSum         669
diasumfact_QMSum          569
fib_cnndm                 543
instrusum_instrusum       458
WiCE                      358
gumsum_gumsum              95
Name: count, dtype: int64


In [25]:
# # Find cases from the test set that are not in the GPT-3.5 predictions

# test_copy = test.copy()

# x = pd.merge(test, gpt, on=['source', 'target'], how='inner')
# x = x[['source','target','label_x','subset_x', 'prediction', 'prob']]
# x = x.rename(columns={'label_x': 'label', 'subset_x': 'subset'})
# # y = pd.merge(gpt, test_copy, on=['source', 'target'], how='left')

# print(len(x))
# # find null in y
# x.isnull().values.any()

In [26]:
# x.sample(2)

In [27]:
def get_gpt_metrics(data):
    # Invert probabilities for cases where prediction is "No"
    def invert_prob(row):
        label = row['label']
        preds = row['prediction']
        label_prob = row['prob']
                    
        if label == 'Yes':
            row['label'] = 1
        if label == 'No':
            row['label'] = 0
        if preds == 'Yes':
            row['prediction'] = 1
        if preds == 'No':
            row['prediction'] = 0              
            row['prob'] = 1 - label_prob
        return row

    data = data.apply(invert_prob, axis=1)
    
    y_true = data['label']
    y_pred = data['prediction']
    y_pred_prob = data['prob']
    
    roc = roc_auc_score(y_true, y_pred_prob)
    
    # return {'accuracy': acc, 'f1': f1, 'roc_auc': roc}
    return roc

In [28]:
# # Deduplicate the data

# import hashlib

# def get_hash(row):
#     source = row['source']
#     target = row['target']
#     row['hash'] = hashlib.md5(f"{source}{target}".encode()).hexdigest()
    
#     return row

# gpt = gpt.apply(get_hash, axis=1)

# # drop duplicates from gpt
# gpt_dedup = gpt.drop_duplicates(subset=['hash'], keep='first')

# print(len(gpt), len(gpt_dedup))
# gpt_dedup.label.value_counts()

In [29]:
# For each subset calculate the metrics
gpt_metrics = gpt.groupby('subset').apply(get_gpt_metrics)
gpt_metrics

subset
ClaimVerify             0.787961
DADC-NLI                0.840711
DeFacto                 0.675185
ExpertQA                0.634823
FactCheck-GPT           0.809871
FoolMeTwice             0.920919
Lfqa                    0.896013
Reveal                  0.942028
Seahorse                0.734978
WiCE                    0.640041
aggrefact_cnndm         0.666128
aggrefact_xsum          0.784158
alisawuffles/WANLI      0.810824
anli                    0.782509
bump_cnndm              0.764673
diasumfact_QMSum        0.701812
diasumfact_SAMSum       0.806191
fib_cnndm               0.655761
fib_xsum                0.767938
gumsum_gumsum           0.823917
halueval_cnndm          0.685213
instrusum_instrusum     0.504676
llm_summaries_cnndm     0.861768
llm_summaries_xsum      0.759703
scitail                 0.903457
tofueval_mediasum       0.737195
tofueval_meetingbank    0.819473
dtype: float64

In [31]:
len(gpt)

62823

### AlignScore

In [32]:
alignscore_base = pd.read_json("../data/eval-data/alignscore_base.json", lines=True)
alignscore_large = pd.read_json("../data/eval-data/alignscore_large.json", lines=True)

In [33]:
alignscore_base.sample(2)

,source,target,label,subset,alignscore_base
9223,"Arsenal goalkeeper Wojciech Szczesny admits he feels sorry for Adam Federici, insisting the Reading stopper was 'the best player on the pitch' despite his costly error in Saturday's FA Cup semi-final. Federici allowed Alexis Sanchez's driven shot to slip through his body in extra-time as Arsenal ran out 2-1 winners at Wembley to reach their second consecutive final, having beaten Hull to lift the trophy last year. The mistake overshadowed a number of strong saves Federici had made to keep his side in the game and Szczesny had only kind words for his opposite number. Wojciech Szczesny (left) says he feels sorry for Adam Federici after his mistake in the FA Cup semi-final. 'He can be very proud. It only went to extra-time because the goalkeeper was magnificent,' Szczesny told Arsenal's o...","Wojciech Szczesny believes Adam Federici played better in the FA Cup semi-final than any other player on the pitch, despite his unfortunate error that cost Reading the game. Arsenal won the match 2-1 in extra-time after Federici allowed Alexis Sanchez to score. Szczesny empathized with Federici and stated that he was hoping to win the match. Arsenal are now one step closer to becoming the most successful FA Cup team of all time.",No,halueval_cnndm,0.850175
58313,"you to view the basic structure and design as an introduction to the parts of an airplane. The parts of a plane are basic knowledge for all pilots. Whether you fly a Cessna 172 Skyhawk or a Boeing 747, pilots must know the main sections and parts of an airplane.The Main Sections of an AirplaneAirplanes are not all alike, but they are comprised of basic components. The main sections of an airplane include the fuselage, wings, cockpit, engine, propeller, tail assembly, and landing gear. Understanding the basic functions of how these parts interact is the first step to understanding the principles of aerodynamics.What is the fuselage?The fuselage is the main section, or body, of the airplane. (If “fuselage” sounds like a French word to you, you’re right. That’s because it is derived from ...","This includes knowledge about the fuselage, wings, cockpit, engine, propeller, tail assembly, and landing gear, as well as the principles of aerodynamics .",Yes,ExpertQA,0.353359


In [34]:
alignscore_large.label.value_counts()

label
Yes    31469
No     31354
Name: count, dtype: int64

In [35]:
# Fix the label column in alignscore_base by copying them from alignscore_large
# First do a merge on the ['source', 'target'] columns

alignscore_base = alignscore_base.merge(alignscore_large[['source', 'target', 'label']], on=['source', 'target'], how='left')

alignscore_base['label_x'] = alignscore_base['label_y']
alignscore_base = alignscore_base.drop(columns=['label_y'])
alignscore_base = alignscore_base.rename(columns={'label_x': 'label'})

alignscore_base.label.value_counts()

label
Yes    31469
No     31354
Name: count, dtype: int64

In [36]:
alignscore_base = alignscore_base.apply(convert_label, axis=1)
alignscore_large = alignscore_large.apply(convert_label, axis=1)

In [37]:
# roc for each subset   
alignscore_base_auc = alignscore_base.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['alignscore_base']))
alignscore_large_auc = alignscore_large.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['alignscore_large']))

alignscore_base_auc, alignscore_large_auc


(subset
 ClaimVerify             0.727478
 DADC-NLI                0.788478
 DeFacto                 0.706138
 ExpertQA                0.573432
 FactCheck-GPT           0.777900
 FoolMeTwice             0.859929
 Lfqa                    0.845584
 Reveal                  0.917330
 Seahorse                0.616925
 WiCE                    0.723930
 aggrefact_cnndm         0.647910
 aggrefact_xsum          0.716962
 alisawuffles/WANLI      0.826411
 anli                    0.741015
 bump_cnndm              0.659820
 diasumfact_QMSum        0.663104
 diasumfact_SAMSum       0.763708
 fib_cnndm               0.140707
 fib_xsum                0.711012
 gumsum_gumsum           0.773540
 halueval_cnndm          0.654022
 instrusum_instrusum     0.530825
 llm_summaries_cnndm     0.748689
 llm_summaries_xsum      0.721820
 scitail                 0.857036
 tofueval_mediasum       0.752660
 tofueval_meetingbank    0.808946
 dtype: float64,
 subset
 ClaimVerify             0.704079
 DADC-NLI      

### UniEval

In [38]:
unieval = pd.read_json("../data/eval-data/unieval.json", lines=True)
unieval.label.value_counts()

label
Yes    31469
No     31354
Name: count, dtype: int64

In [39]:
unieval.sample(2)

,source,target,label,subset,unieval
11989,"The number of Chinese tourists eager to travel the world has risen dramatically over the last few years, and Beijing International Airport is launching the world's biggest terminal to cope with the substantial increase. Opening in 2018, the gigantic Terminal 1 will cover 700,000 square metres, and is set to handle 45million passengers a year. Renown British-Iraqi architect, Zaha Hadid, has collaborated with airport developers ADPI to create a six-tier concept which aims to decrease customer walking distances, and increase connectivity. Scroll down for video. Terminal 1 at Beijing International Airport is set to the largest in the world, in order to cope with the surge in tourist numbers. Aiming to be the perfect welcome to Beijing, the designs focus on creating a feeling of space and o...",China's tourism industry is set to reach new heights with the world's largest airport terminal opening in Beijing in 2018. The terminal will handle 45 million passengers per year and will be designed to create a feeling of openness and space. Zaha Hadid collaborated with airport developers ADPI to bring the concept to life.,No,halueval_cnndm,0.967260
38944,The need for a national program to develop a common language for a new information age was recognized by the Congress in the National Information Infrastructure Act of 1996.,The need for a national program to develop a common language for a new information age was recognized by the Congress.,Yes,alisawuffles/WANLI,0.907713


In [40]:
unieval = unieval.apply(convert_label, axis=1)

roc = roc_auc_score(unieval['label'], unieval['unieval'])

print("AUC ROC score: ", roc)

# Subset wise AUC ROC score
unieval_auc = unieval.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['unieval']))
unieval_auc

AUC ROC score:  0.7017943036725703


subset
ClaimVerify             0.767457
DADC-NLI                0.815664
DeFacto                 0.564717
ExpertQA                0.597975
FactCheck-GPT           0.765852
FoolMeTwice             0.880201
Lfqa                    0.832551
Reveal                  0.915155
Seahorse                0.679787
WiCE                    0.675967
aggrefact_cnndm         0.704667
aggrefact_xsum          0.634889
alisawuffles/WANLI      0.772040
anli                    0.677980
bump_cnndm              0.737285
diasumfact_QMSum        0.617346
diasumfact_SAMSum       0.646615
fib_cnndm               0.182047
fib_xsum                0.678497
gumsum_gumsum           0.545669
halueval_cnndm          0.638827
instrusum_instrusum     0.597126
llm_summaries_cnndm     0.783651
llm_summaries_xsum      0.676358
scitail                 0.837040
tofueval_mediasum       0.657103
tofueval_meetingbank    0.808075
dtype: float64

### QuestEval

In [41]:
questeval = pd.read_json("../data/eval-data/questeval.json", lines=True)
print(len(questeval))
print(questeval.columns)
print(questeval.subset.value_counts())
print(questeval.label.value_counts())
questeval.sample(2)

62823
Index(['source', 'target', 'label', 'subset', 'questeval'], dtype='object')
subset
halueval_cnndm          19998
alisawuffles/WANLI       5000
Seahorse                 4135
ExpertQA                 3702
fib_xsum                 3534
anli                     3200
scitail                  2126
Lfqa                     1911
DeFacto                  1836
llm_summaries_cnndm      1829
llm_summaries_xsum       1726
Reveal                   1705
FactCheck-GPT            1565
FoolMeTwice              1379
aggrefact_xsum           1335
ClaimVerify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
DADC-NLI                  766
tofueval_mediasum         725
diasumfact_SAMSum         669
diasumfact_QMSum          569
fib_cnndm                 543
instrusum_instrusum       458
WiCE                      358
gumsum_gumsum              95
Name: count, dtype: int64
label
Yes    31469
No     31354
Name: count, dtype: int64


,source,target,label,subset,questeval
13796,"Her style is followed by women across the globe and whatever she wears is sure to sell out within minutes. Yet the Duchess of Cambridge has been dismissed as an ‘uneventful’ dresser by leading author Margaret Atwood. And most controversially, the 75-year-old said Kate hasn’t lived up to the fashion icon reputation of her husband’s late mother, Princess Diana. Scroll down for video. The author said Kate (pictured right) hasn’t lived up to the fashion icon reputation of Princess Diana (left) Miss Atwood said she thinks the duchess is cautious when it comes to clothes and is told what to wear by advisers. But she added she is right not to follow in Diana’s footsteps. ‘I think she dresses quite uneventfully,’ the author of The Handmaid’s Tale said. ‘I think she’s watching her back, I think...",Author Margaret Atwood dismissed Katherine as an 'uneventful' dresser. She says Duchess of Cambridge hasn't lived up to fashion icon Diana. Miss Atwood says Kate is cautious when it comes to clothing.\n,Yes,halueval_cnndm,0.508444
7795,"A youngster admiring his catch was left surprised when the fish jumped up and slapped him in the face. Quinn Patrick was on a fishing trip with his dad at Snow Lake, Indiana when the pair caught a bowfin, videoed lying lifeless on a concrete dock. Crouching over the fish, Quinn deliberates whether someone should put it back in the water. When suddenly the Bowfin propels itself from the ground and slaps the youngster straight in the face with its large tail. The sound of the fish making impact with the youngster’s face is not dissimilar to a sound effect used in a cartoon. Recoiling, Quinn stumbles backwards in shock as his dad begins laughing – prompting the youngster to laugh along with him. Quinn Patrick was on a fishing trip with his dad at Snow Lake, Indiana when the pair caught a ...","A brave young boy catches a Bowfin fish while on a fishing trip with his dad, but is surprised when it jumps up and gives him a high five with its tail. The fish is later found to have magical powers and has been labeled as a rare species.",No,halueval_cnndm,0.449519


In [42]:
questeval = questeval.apply(convert_label, axis=1)
print(questeval.label.value_counts())

# Calculate metrics for questeval
questeval_roc = roc_auc_score(questeval['label'], questeval['questeval'])
print(f"ROC AUC: {questeval_roc}")

# roc for each subset
questeval_auc = questeval.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['questeval']))

questeval_auc

label
1    31469
0    31354
Name: count, dtype: int64
ROC AUC: 0.6240841791239211


subset
ClaimVerify             0.712643
DADC-NLI                0.568126
DeFacto                 0.645085
ExpertQA                0.593886
FactCheck-GPT           0.754832
FoolMeTwice             0.684465
Lfqa                    0.806550
Reveal                  0.849062
Seahorse                0.631419
WiCE                    0.599519
aggrefact_cnndm         0.714111
aggrefact_xsum          0.610757
alisawuffles/WANLI      0.623556
anli                    0.480432
bump_cnndm              0.620373
diasumfact_QMSum        0.569502
diasumfact_SAMSum       0.568736
fib_cnndm               0.342142
fib_xsum                0.611612
gumsum_gumsum           0.793785
halueval_cnndm          0.580613
instrusum_instrusum     0.540282
llm_summaries_cnndm     0.872060
llm_summaries_xsum      0.772084
scitail                 0.926951
tofueval_mediasum       0.663663
tofueval_meetingbank    0.706323
dtype: float64

### MiniCheck

In [43]:
minicheck = pd.read_json("../data/eval-data/minicheck.json", lines=True)
print(len(minicheck))
print(minicheck.columns)
print(minicheck.label.value_counts())
print(minicheck.subset.value_counts())

62823
Index(['source', 'target', 'label', 'subset', 'minicheck'], dtype='object')
label
Yes    31469
No     31354
Name: count, dtype: int64
subset
halueval_cnndm          19998
alisawuffles/WANLI       5000
Seahorse                 4135
ExpertQA                 3702
fib_xsum                 3534
anli                     3200
scitail                  2126
Lfqa                     1911
DeFacto                  1836
llm_summaries_cnndm      1829
llm_summaries_xsum       1726
Reveal                   1705
FactCheck-GPT            1565
FoolMeTwice              1379
aggrefact_xsum           1335
ClaimVerify              1087
aggrefact_cnndm          1017
bump_cnndm                785
tofueval_meetingbank      770
DADC-NLI                  766
tofueval_mediasum         725
diasumfact_SAMSum         669
diasumfact_QMSum          569
fib_cnndm                 543
instrusum_instrusum       458
WiCE                      358
gumsum_gumsum              95
Name: count, dtype: int64


In [44]:
minicheck = minicheck.apply(convert_label, axis=1)

roc = roc_auc_score(minicheck['label'], minicheck['minicheck'])

print("AUC ROC score: ", roc)

# Subset wise AUC ROC score
minicheck_auc = minicheck.groupby('subset').apply(lambda x: roc_auc_score(x['label'], x['minicheck']))
minicheck_auc

AUC ROC score:  0.7500985604208029


subset
ClaimVerify             0.810586
DADC-NLI                0.835994
DeFacto                 0.749103
ExpertQA                0.617747
FactCheck-GPT           0.841101
FoolMeTwice             0.868752
Lfqa                    0.931178
Reveal                  0.938783
Seahorse                0.698619
WiCE                    0.767772
aggrefact_cnndm         0.732931
aggrefact_xsum          0.768704
alisawuffles/WANLI      0.831861
anli                    0.774468
bump_cnndm              0.692243
diasumfact_QMSum        0.734158
diasumfact_SAMSum       0.777320
fib_cnndm               0.469333
fib_xsum                0.767545
gumsum_gumsum           0.784369
halueval_cnndm          0.672593
instrusum_instrusum     0.561546
llm_summaries_cnndm     0.813549
llm_summaries_xsum      0.772313
scitail                 0.889745
tofueval_mediasum       0.810905
tofueval_meetingbank    0.848237
dtype: float64

### LLaMA-3

In [45]:
llama3_zs = pd.read_json("../data/eval-data/llama3-8b.jsonl", lines=True)
llama3_ft = pd.read_json("../data/eval-data/llama3-8b-finetune.jsonl", lines=True)
print(len(llama3_zs), len(llama3_ft))

# Deduplicate based on the ['source', 'target'] columns

llama3_zs = llama3_zs.drop_duplicates(subset=['source', 'target'])
llama3_ft = llama3_ft.drop_duplicates(subset=['source', 'target'])
print(len(llama3_zs), len(llama3_ft))

# # Deduplicate based on hashing
# import hashlib

# def get_hash(row):
#     source = row['source']
#     target = row['target']
#     subset = row['subset']
#     try:
#         hash_val = hashlib.md5(f"{source}{target}".encode()).hexdigest()
#     except:
#         print(subset)
#         print(inp)
#     row['hash'] = hash_val
    
#     return row

# llama3_zs = llama3_zs.apply(get_hash, axis=1)
# llama3_ft = llama3_ft.apply(get_hash, axis=1)
# llama3_zs = llama3_zs.drop_duplicates(subset=['hash'])
# llama3_ft = llama3_ft.drop_duplicates(subset=['hash'])
# print(len(llama3_zs), len(llama3_ft))

print(llama3_zs.label.value_counts(), llama3_zs.prediction.value_counts())
print(llama3_ft.label.value_counts(), llama3_ft.prediction.value_counts())
llama3_zs.sample(2)

62823 62821
62823 62821
label
Yes    31469
No     31354
Name: count, dtype: int64 prediction
Yes    43929
No     18894
Name: count, dtype: int64
label
Yes    31469
No     31352
Name: count, dtype: int64 prediction
Yes    38846
No     23975
Name: count, dtype: int64


,source,target,label,subset,prediction,prob
3334,"Police said that someone opened fire from a passing car and witnesses spoke of up to a dozen shots being fired close to the main police station. ""Nothing is known about the culprits yet but we can guess that this involves local crime gangs,"" investigators told German media. The suspects fled and the area was quickly cordoned off. The shots were fired in Friedrich-Stoltze Square, a busy area of bars and cafes, as local people enjoyed the Ascension Day holiday in the sunshine.",Shots have been fired near a police station in the German city of Stuttgart.,No,aggrefact_xsum,Yes,0.988176
18576,"Amargosa is an unincorporated community and census-designated place in Jim Wells County, Texas, United States. Its population was 291 as of the 2010 census. Prior to 2010, the community was grouped with nearby Owl Ranch as part of the Owl Ranch-Amargosa census-designated place. The community is named for the Amargosa Creek that runs nearby. The word ""amargosa"" means ""bitter"" in Spanish.","Amargosa is a place in Jum Wells County, Texas that has a large population.",No,anli,No,0.000152


In [46]:
llama3_ft.sample(2)

,source,target,label,subset,prediction,prob
34373,"Midfielder Cheikhou Kouyate has warned West Ham must be ready to face a 'wounded animal' at Manchester City on Sunday, but will be ready to inflict more pain on Manuel Pellegrini's men. The Barclays Premier League champions have seen their title defence left in tatters following a run of four defeats in six matches, thrashed 4-2 at rivals United last weekend - a result which left the long-term future of manager Pellegrini in doubt as City might now face a scrap with Liverpool for a Champions League qualifying spot. West Ham are also in need a pick-me-up, having seen victory slip through their fingers when Stoke netted a stoppage time equaliser at Upton Park. West Ham midfielder Cheikhou Kouyate is wary of facing champions Manchester City on Sunday. City midfielder Yaya Toure (centre) a...","West Ham face Manchester City at the Etihad on Sunday, KO at 1.30pm. Cheikhou Kouyate believes the Hammers must not underestimate City. Manuel Pellegrini's side have lost their last two Premier League games. Sam Allardyce wants West Ham to be defensively solid on Sunday.\n",Yes,halueval_cnndm,Yes,0.573691
54658,"By\n- Judith Martin\n- Nicholas Ivor Martin and Jacobina Martin\nDEAR MISS MANNERS: I teach at a small college where, before COVID-19, I would regularly meet prospective students and their parents in my office on campus.\nOccasionally these parents are prominent in politics, and given the deep rifts in our political culture these days, I wonder how I should treat a parent whose positions are abhorrent to me.\nI would welcome the student just as I would welcome anyone, and I would greet the parent distantly but politely, since while on campus I try to be nonpolitical. But if that parent were to extend a hand, can I -- and should I -- refuse to take it, perhaps with a polite “I can’t shake your hand”?\nGENTLE READER: If it would make you feel virtuous to do so -- and provided you do not ...",It is important to examine shifting beliefs and values through cultural history and consider the ethical implications of supporting people with abhorrent views.,No,ClaimVerify,No,0.076956


In [47]:
# convert "Yes" to 1 and "No" to 0
def convert_label(row):
    if row['label'] == 'Yes':
        row['label'] = 1
    if row['label'] == 'No':
        row['label'] = 0
    return row

llama3_zs = llama3_zs.apply(convert_label, axis=1)
print(llama3_zs.label.value_counts())

llama3_ft = llama3_ft.apply(convert_label, axis=1)
print(llama3_ft.label.value_counts())

label
1    31469
0    31354
Name: count, dtype: int64
label
1    31469
0    31352
Name: count, dtype: int64


In [48]:
# Calculate metrics for llama3_zs
llama3_zs_roc = roc_auc_score(llama3_zs['label'], llama3_zs['prob'])
print(f"ROC AUC: {llama3_zs_roc}")
llama3_ft_roc = roc_auc_score(llama3_ft['label'], llama3_ft['prob'])
print(f"ROC AUC: {llama3_ft_roc}")

print()  

# roc for each subset
for subset in llama3_zs['subset'].unique():
    subset_data = llama3_zs[llama3_zs['subset'] == subset]
    try:
        roc = roc_auc_score(subset_data['label'], subset_data['prob'])
    except Exception as e:
        roc = 0
        # print(subset_data)
    print(f"{subset}: {roc}")

print()  
    
for subset in llama3_ft['subset'].unique():
    subset_data = llama3_ft[llama3_ft['subset'] == subset]
    try:
        roc = roc_auc_score(subset_data['label'], subset_data['prob'])
    except Exception as e:
        roc = 0
        # print(subset_data)
    print(f"{subset}: {roc}")


ROC AUC: 0.7170519503877647
ROC AUC: 0.7800030927531358

aggrefact_cnndm: 0.6291189865666501
fib_cnndm: 0.889180182699478
Seahorse: 0.6897540752240492
halueval_cnndm: 0.7059943888777754
llm_summaries_cnndm: 0.8532925212624931
alisawuffles/WANLI: 0.755825360630206
DeFacto: 0.5943360748612956
DADC-NLI: 0.8287257967198728
llm_summaries_xsum: 0.6979800769566066
anli: 0.683731078057128
bump_cnndm: 0.7694952439850399
fib_xsum: 0.7078539881344759
scitail: 0.8758842616230456
tofueval_meetingbank: 0.7893494623655914
diasumfact_SAMSum: 0.7998652421762904
WiCE: 0.6890615311667944
FoolMeTwice: 0.928601332104734
diasumfact_QMSum: 0.726707639843233
tofueval_mediasum: 0.7763625467849784
instrusum_instrusum: 0.5867424242424243
gumsum_gumsum: 0.7923728813559323
aggrefact_xsum: 0.7329510758288732
Lfqa: 0.794342189952461
ExpertQA: 0.5817252593584772
ClaimVerify: 0.666626262110734
FactCheck-GPT: 0.7752413524685504
Reveal: 0.9110174053127151

Seahorse: 0.742920702834496
halueval_cnndm: 0.6951997599519903
D

In [49]:
# calculate balanced accuracy on LLM-AggreFact for llama3_zs and llama3_ft

llama3_zs_llm_aggrefact = llama3_zs[llama3_zs['subset'].isin(subsets_llm_aggrefact)]
llama3_ft_llm_aggrefact = llama3_ft[llama3_ft['subset'].isin(subsets_llm_aggrefact)]
x1_aggrefact = llama3_zs[llama3_zs['subset'].isin(["aggrefact_cnndm", "aggrefact_xsum"])]
x2_aggrefact = llama3_ft[llama3_ft['subset'].isin(["aggrefact_cnndm", "aggrefact_xsum"])]

print(llama3_zs_llm_aggrefact.label.value_counts(), llama3_zs_llm_aggrefact.prediction.value_counts())
print(x1_aggrefact.label.value_counts(), x1_aggrefact.prediction.value_counts())
print(len(x1_aggrefact), len(y_aggrefact))

print(llama3_ft_llm_aggrefact.prediction.value_counts(), llama3_ft_llm_aggrefact.label.value_counts())
print(x2_aggrefact.label.value_counts(), x2_aggrefact.prediction.value_counts())
print(len(x2_aggrefact), len(y_aggrefact))

y1 = y_aggrefact.merge(x1_aggrefact[['source', 'target', 'prediction', 'prob']], on=['source', 'target'], how='left')
y2 = y_aggrefact.merge(x2_aggrefact[['source', 'target', 'prediction', 'prob']], on=['source', 'target'], how='left')
print(len(y1), len(y2))
print(y1.label.value_counts(), y1.prediction.value_counts())
print(y2.label.value_counts(), y2.prediction.value_counts())

llama3_zs_llm_aggrefact = pd.concat([llama3_zs_llm_aggrefact, y1])
llama3_ft_llm_aggrefact = pd.concat([llama3_ft_llm_aggrefact, y2])

def convert_prediction(row):
    if row['prediction'] == 'Yes':
        row['prediction'] = 1
    if row['prediction'] == 'No':
        row['prediction'] = 0
    return row

llama3_zs_llm_aggrefact = llama3_zs_llm_aggrefact.apply(convert_prediction, axis=1)
llama3_zs_llm_aggrefact = llama3_zs_llm_aggrefact.apply(convert_label, axis=1)

llama3_ft_llm_aggrefact = llama3_ft_llm_aggrefact.apply(convert_prediction, axis=1)
llama3_ft_llm_aggrefact = llama3_ft_llm_aggrefact.apply(convert_label, axis=1)

# llama3_ft_llm_aggrefact.sample(2)

label
1    6939
0    4884
Name: count, dtype: int64 prediction
Yes    8199
No     3624
Name: count, dtype: int64
label
1    1580
0     772
Name: count, dtype: int64 prediction
Yes    2158
No      194
Name: count, dtype: int64
2352 1116
prediction
Yes    6381
No     5442
Name: count, dtype: int64 label
1    6939
0    4884
Name: count, dtype: int64
label
1    1580
0     772
Name: count, dtype: int64 prediction
Yes    1723
No      629
Name: count, dtype: int64
2352 1116
1116 1116
label
Yes    786
No     330
Name: count, dtype: int64 prediction
Yes    1031
No       85
Name: count, dtype: int64
label
Yes    786
No     330
Name: count, dtype: int64 prediction
Yes    844
No     272
Name: count, dtype: int64


In [50]:
# print label and prediction value counts
print(llama3_zs_llm_aggrefact.label.value_counts())
print(llama3_zs_llm_aggrefact.prediction.value_counts())
print(llama3_ft_llm_aggrefact.label.value_counts())
print(llama3_ft_llm_aggrefact.prediction.value_counts())

label
1    7725
0    5214
Name: count, dtype: int64
prediction
1    9230
0    3709
Name: count, dtype: int64
label
1    7725
0    5214
Name: count, dtype: int64
prediction
1    7225
0    5714
Name: count, dtype: int64


In [51]:
# Print balanced accuracy for each subset
s = 0
n = 0
for subset in llama3_zs_llm_aggrefact.subset.unique():
    subset_data = llama3_zs_llm_aggrefact[llama3_zs_llm_aggrefact['subset'] == subset]
    balanced_acc = balanced_accuracy_score(subset_data['label'], subset_data['prediction'])
    if balanced_acc > 0:
        print(f"{subset}: {balanced_acc}")
        s += balanced_acc
        n+=1
    else:
        print(f"{subset}: {0}")
avg_balanced_acc = s / n
print(f"Average balanced accuracy: {avg_balanced_acc}")
print()


s = 0
n = 0
for subset in llama3_ft_llm_aggrefact.subset.unique():
    subset_data = llama3_ft_llm_aggrefact[llama3_ft_llm_aggrefact['subset'] == subset]
    balanced_acc = balanced_accuracy_score(subset_data['label'], subset_data['prediction'])
    if balanced_acc > 0:
        print(f"{subset}: {balanced_acc}")
        s += balanced_acc
        n+=1
    else:
        print(f"{subset}: {0}")
    
avg_balanced_acc = s / n
print(f"Average balanced accuracy: {avg_balanced_acc}")

tofueval_meetingbank: 0.7194623655913979
WiCE: 0.5710508078929132
tofueval_mediasum: 0.7202310862525758
Lfqa: 0.6448243543852121
ExpertQA: 0.5430426176247272
ClaimVerify: 0.60490298653465
FactCheck-GPT: 0.7053710430721328
Reveal: 0.8167847654492816
AggreFact-CNN: 0.5486395629793046
AggreFact-XSum: 0.5907460960092539
Average balanced accuracy: 0.646505568579145

tofueval_meetingbank: 0.7277956989247312
tofueval_mediasum: 0.7097018377560032
WiCE: 0.7131524236787394
ExpertQA: 0.581899538677807
Reveal: 0.8652414713198741
Lfqa: 0.8178530697049424
FactCheck-GPT: 0.7740748528174937
ClaimVerify: 0.7028946674492391
AggreFact-CNN: 0.5386595230591449
AggreFact-XSum: 0.7176980913823019
Average balanced accuracy: 0.7148971174770277


### Mean Win Rate

In [52]:
# Taken from the HELM implementation
# https://github.com/stanford-crfm/helm/blob/d5a43ad3572a7e507533f1f07863aa2033fa6dc9/src/helm/benchmark/presentation/summarize.py#L217

from statistics import mean, median
from typing import List, Optional

# Data for each method across all benchmarks and tables
# Scores from the four tables for different benchmarks and metrics
rouge_scores = [
    51.3, 75.2, 52.4, 45.3, 50.0, 44.1, 50.9, 50.9, 74.3, 45.7,  # Table 1
    69.1, 51.9, 4.7, 85.3, 42.5, 47.1, 38.7, 60.1,               # Table 2
    56.2, 57.2, 56.4, 67.3,                                      # Table 3
    60.8, 56.8, 77.3, 51.5, 53.3                                 # Table 4
]

bertscore_scores = [
    55.6, 80.1, 57.5, 42.5, 55.9, 43.8, 57.4, 56.4, 75.9, 52.3,  # Table 1
    71.3, 54.4, 12.4, 88.7, 53.2, 55.2, 54.4, 68.4,              # Table 2
    66.1, 63.3, 70.6, 68.5,                                      # Table 3
    66.3, 72.7, 85.6, 55.1, 70.5                                 # Table 4
]

bartscore_scores = [
    57.8, 87.6, 60.6, 45.2, 63.5, 49.7, 58.4, 58.9, 68.9, 57.1,  # Table 1
    65.7, 58.3, 2.1, 87.2, 38.9, 71.8, 44.2, 72.7,               # Table 2
    66.7, 63.3, 68.3, 71.2,                                      # Table 3
    79.3, 70.4, 87.8, 57.9, 77.7                                 # Table 4
]

questeval_scores = [
    62.4, 92.7, 68.4, 48.0, 60.0, 56.8, 64.5, 63.1, 79.4, 54.0,  # Table 1
    71.4, 62.0, 34.2, 87.2, 58.1, 61.1, 61.2, 77.2,              # Table 2
    57.0, 56.9, 66.4, 70.6,                                      # Table 3
    84.9, 75.5, 80.7, 59.4, 71.3                                 # Table 4
]

alignscore_base_scores = [
    82.6, 85.7, 86.0, 74.1, 72.4, 78.8, 70.6, 61.7, 77.4, 53.1,  # Table 1
    64.8, 66.0, 14.1, 74.9, 65.4, 71.7, 71.1, 72.2,              # Table 2
    66.3, 76.4, 75.3, 80.9,                                      # Table 3
    91.7, 77.8, 84.6, 57.3, 72.7                                 # Table 4
]

alignscore_large_scores = [
    83.9, 88.1, 83.6, 79.3, 77.9, 87.0, 71.4, 65.1, 74.4, 59.6,  # Table 1
    55.8, 74.4, 13.2, 78.7, 73.2, 72.4, 78.2, 72.6,              # Table 2
    67.4, 78.2, 74.1, 80.6,                                      # Table 3
    92.2, 81.4, 82.2, 57.7, 70.4                                 # Table 4
]

gpt_35_turbo_scores = [
    81.5, 90.3, 92.2, 80.0, 64.0, 84.1, 67.5, 73.5, 82.4, 50.5,  # Table 1
    66.6, 80.3, 65.7, 86.5, 68.2, 78.4, 76.4, 76.0,              # Table 2
    69.6, 82.7, 73.7, 82.0,                                      # Table 3
    94.2, 81.0, 89.6, 63.5, 78.8                                 # Table 4
]

llama_3_8b_scores = [
    75.6, 87.6, 92.9, 68.4, 68.9, 82.9, 59.4, 69.0, 79.2, 58.7,  # Table 1
    66.0, 76.9, 89.9, 85.3, 70.6, 73.3, 70.8, 69.8,              # Table 2
    72.7, 80.0, 77.6, 78.9,                                      # Table 3
    91.1, 77.5, 79.4, 58.2, 66.7                                 # Table 4
]

minicheck_scores = [
    83.2, 88.9, 86.9, 77.4, 76.8, 83.6, 74.9, 69.9, 78.4, 56.2,  # Table 1
    67.3, 69.2, 46.9, 81.4, 67.3, 76.9, 76.8, 77.2,              # Table 2
    73.4, 77.7, 81.1, 84.8,                                      # Table 3
    93.9, 84.1, 93.1, 61.8, 81.1                                 # Table 4
]


flan_t5_b_ft_scores = [
    87.7, 98.5, 91.9, 75.4, 86.6, 86.1, 77.3, 72.1, 81.5, 57.4,  # Table 1
    68.5, 68.6, 27.2, 84.3, 59.8, 74.1, 82.1, 75.7,              # Table 2
    70.0, 75.7, 78.1, 76.6,                                      # Table 3
    90.0, 82.3, 88.5, 59.5, 79.9                                 # Table 4
]

flan_t5_l_ft_scores = [
    89.0, 99.3, 94.0, 83.4, 88.8, 89.9, 80.8, 74.4, 82.7, 55.7,  # Table 1
    69.7, 76.9, 54.6, 85.4, 67.4, 74.8, 87.6, 75.9,              # Table 2
    62.3, 75.9, 75.4, 75.9,                                      # Table 3
    90.5, 84.0, 88.1, 61.5, 80.0                                 # Table 4
]

llama_3_8b_ft_scores = [
    87.2, 98.2, 95.8, 85.4, 78.7, 89.8, 69.1, 74.3, 83.9, 63.9,  # Table 1
    66.0, 70.7, 47.4, 77.7, 69.5, 75.8, 76.5, 76.2,              # Table 2
    71.3, 83.2, 75.3, 82.9,                                      # Table 3
    93.8, 86.3, 90.7, 61.7, 75.7                                 # Table 4
]

# Combine all scores into a list of lists
data_scores = [
    rouge_scores,             # ROUGE
    bertscore_scores,         # BERTScore
    bartscore_scores,         # BARTScore
    questeval_scores,         # QuestEval
    alignscore_base_scores,   # AlignScore-B
    alignscore_large_scores,  # AlignScore-L
    gpt_35_turbo_scores,      # gpt-3.5-turbo
    llama_3_8b_ft_scores,     # LLaMa-3-8B (FT)
    minicheck_scores,         # Minicheck
    llama_3_8b_scores,        # LLaMa 3-8B
    flan_t5_b_ft_scores,      # Flan-T5-B (FT)
    flan_t5_l_ft_scores       # Flan-T5-L (FT)
]



def compute_aggregate_row_win_rates(data: List[List[float]], aggregation: str = "mean") -> List[Optional[float]]:
    win_rates_per_row: List[List[float]] = [[] for _ in data] # each row is a method

    # Transpose the data to work with columns
    data_transposed = list(zip(*data))
    
    # print(data_transposed)

    for values in data_transposed:  # each column is a dataset
        # print(values)
        
        x = [(row, j) for j, row in enumerate(values)]
        
        # print(x)
        # print(sorted(x))
        
        for wins, (v, j) in enumerate(sorted(x)): # here j is the index of the method (stored as a row)
            win_rate = wins / (len(values) - 1) # normalize to [0, 1]
            win_rates_per_row[j].append(win_rate) # add to the list of scores for a method (stored as a row)
            
        # print()
        
    aggregate_win_rates: List[Optional[float]] = []
    for win_rates in win_rates_per_row:
        if len(win_rates) == 0:
            aggregate_win_rates.append(None)
        else:
            aggregate = mean(win_rates) if aggregation == "mean" else median(win_rates)
            aggregate_win_rates.append(aggregate)

    return aggregate_win_rates

# Compute median win rates
# median_win_rates = compute_aggregate_row_win_rates(data_scores, "median")
mean_win_rates = compute_aggregate_row_win_rates(data_scores, "mean")
# print(median_win_rates)

# Print mean win rates for each method
methods = [
    "ROUGE",
    "BERTScore",
    "BARTScore",
    "QuestEval",
    "AlignScore-B",
    "AlignScore-L",
    "gpt-3.5-turbo",
    "LLaMa-3-8B (FT)",
    "Minicheck",
    "LLaMa 3-8B",
    "Flan-T5-B (FT)",
    "Flan-T5-L (FT)"
]

# Get rank of each method
def get_rank(data):
    return sorted(range(len(data)), key=lambda k: data[k], reverse=True)

rank = get_rank(mean_win_rates)

# for method, win_rate in zip(methods, mean_win_rates):
#     print(f"{method}: {win_rate}")
    
# print rank, method and win rate
for i, r in enumerate(rank):
    print(f"{i+1}. {methods[r]}: {mean_win_rates[r]}")

1. LLaMa-3-8B (FT): 0.7811447811447811
2. Flan-T5-L (FT): 0.7643097643097643
3. Minicheck: 0.7239057239057239
4. gpt-3.5-turbo: 0.6936026936026936
5. Flan-T5-B (FT): 0.6599326599326599
6. AlignScore-L: 0.531986531986532
7. LLaMa 3-8B: 0.531986531986532
8. AlignScore-B: 0.3939393939393939
9. QuestEval: 0.3737373737373737
10. BARTScore: 0.26936026936026936
11. BERTScore: 0.20875420875420875
12. ROUGE: 0.06734006734006734


In [53]:
# Mean scores for each method

mean_scores = [mean(scores) for scores in data_scores]
for method, score in zip(methods, mean_scores):
    print(f"{method}: {score}")

ROUGE: 54.67777777777778
BERTScore: 61.26296296296296
BARTScore: 62.63703703703704
QuestEval: 66.08888888888889
AlignScore-B: 71.31851851851852
AlignScore-L: 73.07407407407408
gpt-3.5-turbo: 77.00740740740741
LLaMa-3-8B (FT): 78.03703703703704
Minicheck: 76.67407407407407
LLaMa 3-8B: 75.08518518518518
Flan-T5-B (FT): 76.12592592592593
Flan-T5-L (FT): 78.66296296296296


In [ ]:
can you also do two  more things:

Firstly multiply mean win rate by 100 (as it is a percentage)

Secondly, add a new column to the table. it's called "Average AUC". i will post the data below:

ROUGE: 54.67777777777778
BERTScore: 63.455555555555556
BARTScore: 62.63703703703704
QuestEval: 66.08888888888889
AlignScore-B: 71.31851851851852
AlignScore-L: 73.07407407407408
gpt-3.5-turbo: 77.06666666666666
LLaMa-3-8B (FT): 78.03703703703704
Minicheck: 76.67407407407407
LLaMa 3-8B: 75.08518518518518
Flan-T5-B (FT): 76.12592592592593
Flan-T5-L (FT): 78.66296296296296

In [9]:
NLI - ANLI, WANLI, DADC-NLI, SciTail, 


Fact Verification/Claim Verification - 

News Summarization Evaluation - AggreFact, 

NameError: name 'NLI' is not defined

In [ ]:
T